In [ ]:
import mne
import os
# os.chdir('C:\\Users\\Esimk_000\\Desktop\\School\\Spring 2018\\Cs59866\\EEG Project\\seniorprojecteeg-master-fork\\preprocessing\\Artifact_Removal')
%run preprocecssing_helpers.ipynb
%run Batch_ArtifactFilter_Epoch.ipynb
%run ../StimCodes.ipynb

import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
# path = 'D:/EEGLAB/'
# eeglab_files = list_eeglab_files(path)

# path = '/sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/'

In [ ]:
# files = os.listdir(path)

In [ ]:
# epoch_files = [x for x in files if '.fif' in x]

In [ ]:
# epoch_files

In [15]:
# epoch_file = os.path.join(path, epoch_files[10])

In [ ]:
# epoch = mne.read_epochs(epoch_file, preload = True)

In [ ]:
# eeglab_files

In [ ]:
# raw = mne.io.read_raw_eeglab(eeglab_files[4],preload = True)

In [ ]:
# os.chdir("F:\\")
# epoch.save(eeglab_files[4].split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)

In [ ]:
# events = find_events(raw)
# eeglab_trials = get_trials(events)
# eeglab_trials[1:3]

In [ ]:
modality_lexicality_event_ids = {
   "AALL" : 112,
   "AALN" : 212,
   "AANL" : 312,
   "AANN" : 412,
   "AVLL" : 512,
   "AVNN" : 612,
   "VALL" : 712,
   "VANN" : 812,
   "VVLL" : 912,
   "VVLN" : 1012,
   "VVNL" : 1112,
   "VVNN" : 1212
}

In [ ]:
def get_epoch_trials(epoch_object):
    #note: third stim-code has a value that is 1 more than the first.
    events = epoch_object.events
    found_first_stim_code = False
    found_second_stim_code = False
    found_third_stim_code = False
    current_trial = []
    trials = []
    for event in events:
        if not found_first_stim_code and not found_second_stim_code and not found_third_stim_code: #still searching for the end of trial
            current_trial.append(event)
            found_first_stim_code = True
            first_stim_code = event
        elif not found_second_stim_code and not found_third_stim_code:
            current_trial.append(event)
            found_second_stim_code = True
        elif not found_third_stim_code and (event[-1] + 1) == first_stim_code[-1]: 
            current_trial.append(event)
            found_third_stim_code = True
            trials.append(current_trial)
            current_trial = []
            found_first_stim_code = False
            found_second_stim_code = False
            found_third_stim_code = False
        elif not found_third_stim_code and (event[-1] + 1) != first_stim_code[-1]:
            # there is no third event, trial is unfinished.
            current_trial = []
            current_trial.append(event)#assuming it's first stim-code...for now
            found_first_stim_code = True
    return trials

In [ ]:
def get_stim_code_epoch_trials(epoch_object):
    events = epoch_object.events
    found_first_stim_code = False
    found_second_stim_code = False
    found_third_stim_code = False
    current_trial = []
    trials = []
    
    for i in range(len(events[:,-1])-2):
        found_third_stim_code = False
        while (not found_third_stim_code):
            stim_code_1 = events[i]
            stim_code_2 = events[i+1]
            stim_code_3 = events[i+2]
#             print(stim_code_1,stim_code_2,stim_code_3)
            
            if stim_code_1[-1] == stim_code_3[-1] + 1:
                trials.append([stim_code_1,stim_code_2,stim_code_3])
                found_third_stim_code = True
            else:
                
                break;
    return trials

In [ ]:
def stim_code_combination_from_trial(stim_code_tuple, stim_combinations):
    """
    Description:
        returns the correct stim-code combination event id 
        example - a trial has events (20, 15, 19), this method will match those values to a 
        stim combination that is defined in the dictionary stim_combinations
    """
    for stim_code in stim_combinations.keys():
        if stim_code_tuple == stim_code:
            return stim_combinations[stim_code]
    return "NaN"

In [ ]:
def convert_epoch_events_to_stim_combinations(epoch_object):
    trials = get_stim_code_epoch_trials(epoch_object)
#     print(trials)
    new_events_master = []
    
    for trial in trials:
#         print(trial)
        trial = np.array(trial)
        stim_codes = trial[:,-1]
        try:
#             print("stim_codes",stim_codes)
            stim_code_tuple = (stim_codes[0],stim_codes[1], stim_codes[2])
#             print("stim_code_tuple",stim_code_tuple)
            stim_combination = stim_code_combination_from_trial(stim_code_tuple, stim_combinations)
#             print("stim_combination",stim_combination)
            new_event_id = modality_lexicality_event_ids[stim_combination]
#             print("New event_id", new_event_id)
            trial[:,-1] = new_event_id
            new_events_master.append(trial)
        except:
            pass
#             print("Error with stim-code: {0}".format(stim_code_tuple))
#     print(new_events_master)
    return np.concatenate([x for x in new_events_master])

In [ ]:
# new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch.copy())
# new_events.shape

In [ ]:
# print(new_events.shape)
# print(epoch.events.shape)

In [ ]:
# checked_events = np.array([x for x in new_events if x[0] in epoch.events[:,0]])

In [ ]:
def epoch_indexes_to_drop(epoch_object, events):
    epoch_events = epoch_object.events
    indicies_to_drop = []
    for i in range(len(epoch_events)):
        if epoch_events[i,0] not in events[:,0]:
            indicies_to_drop.append(i)
    return indicies_to_drop

In [ ]:
import itertools
def generate_indicies_tuples(indicies_to_drop, len_epochs):
    indicides_tuples = []
    if indicies_to_drop !=[]:
        
        for i in range(len(indicies_to_drop)):
            if i == 0:
                print("i ==0")
                indicides_tuples.append((0,indicies_to_drop[i]))
                print(indicides_tuples)
            else:
                if indicies_to_drop[i-1] +1 == indicies_to_drop[i]:
                    pass
    #                 indicides_tuples.append((indicies_to_drop[i-1] +1, indicies_to_drop[i] +1 ))

                else:
                    indicides_tuples.append((indicies_to_drop[i-1] +1, indicies_to_drop[i]))
            print(indicies_to_drop)

        indicides_tuples.append((indicies_to_drop[-1] +1, len_epochs))
        return indicides_tuples
    else:
        return []
        
def drop_bad_epoch_events(epoch_object, indicies_to_drop):
    indicies_tuples = generate_indicies_tuples(indicies_to_drop, epoch.events.shape[0])
    print("indicides tuple", indicies_tuples)
    epoch_list = []
    if indicies_tuples !=[]:
        for index in indicies_tuples:
            epoch_list.append(epoch_object[index[0]:index[1]])
        if epoch_list != []:
            return mne.epochs.concatenate_epochs(epoch_list)
    else: 
        return epoch_object
#     else:
#         return epoch_object

In [20]:
def convert_epoch_object_events(epoch_object):
    new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch_object.copy())
    indicies_to_drop = epoch_indexes_to_drop(epoch_object.copy(), new_events)
    new_epoch = drop_bad_epoch_events(epoch_object.copy(), indicies_to_drop=indicies_to_drop)
#     print("**", new_events)
#     new_epoch.events = new_events
    return new_epoch

In [16]:
# for epoch_file in epoch_files:
#     file = os.path.join(path, epoch_file)
#     epoch = mne.read_epochs(file, preload = True);
    
#     print("**epoch events shape", epoch.events.shape)
# #     try:
        
#     new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch.copy())
#     print("**new_events shape", new_events.shape)

#     indicies_to_drop = epoch_indexes_to_drop(epoch.copy(), new_events)
#     print("**indicies to drop", indicies_to_drop)
#     new_epoch = drop_bad_epoch_events(epoch.copy(), indicies_to_drop=indicies_to_drop)
#     print("**new epoch shape", new_epoch.get_data().shape)
# #     except:
# #         print("**Error with file {0}".format(epoch_file))

In [9]:
# epoch.events[597:605]

In [10]:
# epoch.events.shape[0]

In [11]:
# indicies_to_drop = epoch_indexes_to_drop(epoch.copy(), new_events)
# new_epoch = drop_bad_epoch_events(epoch.copy(), indicies_to_drop=indicies_to_drop)

In [12]:
# new_epoch.events.shape

In [13]:
# epoch.events = new_events


In [14]:
# new_epochs = mne.epochs.concatenate_epochs([epoch[:1116], epoch[1117:]])

In [ ]:
def isCorrect_stim_code_trials(trials):
    """
    Description:
        A sanity check that makes sure all trials are in the correct order
        The third stim-code needs to have a value that is one less than the 
        first stim-code.
        
    Variables:
        trials: An ndarray of shape  (n_trials, 3, 3)
    -----
    returns:
        boolean if all are correct, or one is incorrect. 
        It will print all incorrect trials
    """
    has_incorrect_trial = True
    for trial in trials:
        if trial[:,-1][0] == trial[:,-1][-1] + 1:
            pass
        else:
            print("Incorrect Trial \n: {0}".format(trial))
            has_incorrect_trial = False
            
                    
    return has_incorrect_trial

In [ ]:
# isCorrect_stim_code_trials(epoch_trials)

In [ ]:
# def create_describer_events(epoch, eeglab):
#     """
#     Description:
#         uses raw eeglab events and epoch events to return a new dictionary
#         with key the timestamp of an event, and value is the new event_id.
#         These event_ids can later be split/manipulated to label data by specific 
#         events
#     Variables:
#         epoch: mne.io.epoch object
#         raw_events: an ndarray of 3d integers shape 
        
#     returns:
#         dictionary
#     """
#     raw_events = find_events(eeglab)
#     raw_trials = get_trials(raw_events)
#     epoch_events = epoch.events
#     epoch_trials = get_epoch_trials(epoch)
    
#     for epoch_trial in epoch_trials:
#         for event in epoch_trial
#             if event[0] in 
#     #  